In [ ]:
import numpy as np
import pandas as pd

In [ ]:
image_csv_path = '/tf/0.DATA/train_image_level.csv'
study_csv_path = '/tf/0.DATA/train_study_level.csv'

image_csv = pd.read_csv(image_csv_path)
study_csv = pd.read_csv(study_csv_path)

In [ ]:
print(image_csv.shape)
print(study_csv.shape)

(6334, 4)
(6054, 5)


In [ ]:
# image 304개 날림
for i in range(len(image_csv)):
    try:
        image_data = image_csv.loc[i]
    except:
        print(i)
        break
    UID = image_data['StudyInstanceUID']
    
    query_st = UID + '_study'
    study_data = study_csv.query("id==@query_st")
    
    # boxes column이 float(nan)인지 확인
    if isinstance(image_data['boxes'], float):
        image_state = 0 # negative
    else:
        image_state = 1 # positive
        
    if study_data['Negative for Pneumonia'].values[0] == 1:
        study_state = 0 # negative
    else:
        study_state = 1 # positive
    
    
    if study_state ^ image_state == 1:
        image_csv = image_csv.drop(i)

In [ ]:
image_csv.reset_index(drop=True, inplace=True)

,index,id,boxes,label,StudyInstanceUID
0,0,000a312787f2_image,"[{'x': 789.28836, 'y': 582.43035, 'width': 102...",opacity 1 789.28836 582.43035 1815.94498 2499....,5776db0cec75
1,1,000c3a3f293f_image,NaN,none 1 0 0 1 1,ff0879eb20ed
2,2,0012ff7358bc_image,"[{'x': 677.42216, 'y': 197.97662, 'width': 867...",opacity 1 677.42216 197.97662 1545.21983 1197....,9d514ce429a7
3,3,001398f4ff4f_image,"[{'x': 2729, 'y': 2181.33331, 'width': 948.000...",opacity 1 2729 2181.33331 3677.00012 2785.33331,28dddc8559b2
4,4,001bd15d1891_image,"[{'x': 623.23328, 'y': 1050, 'width': 714, 'he...",opacity 1 623.23328 1050 1337.23328 2156 opaci...,dfd9fdd85a3e
...,...,...,...,...,...
6025,6329,ffcc6edd9445_image,NaN,none 1 0 0 1 1,7e6c68462e06
6026,6330,ffd91a2c4ca0_image,NaN,none 1 0 0 1 1,8332bdaddb6e
6027,6331,ffd9b6cf2961_image,"[{'x': 2197.38566, 'y': 841.07361, 'width': 31...",opacity 1 2197.38566 841.07361 2513.80265 1292...,7eed9af03814
6028,6332,ffdc682f7680_image,"[{'x': 2729.27083, 'y': 332.26044, 'width': 14...",opacity 1 2729.27083 332.26044 4225.52099 2936...,a0cb0b96fb3d


In [ ]:
# study 중에 매칭되는 image가 전부 삭제되어 없는 경우 study도 삭제
# study 84개 날림
for i in range(len(study_csv)):
    study_data = study_csv.loc[i]
    query_st = study_data['id'].split('_')[0]
    
    image_data = image_csv.query("StudyInstanceUID==@query_st")
    if image_data.empty:
        study_csv = study_csv.drop(i)
print(study_csv.shape)

(5970, 5)


In [ ]:
print(study_csv['id'].nunique())
print(image_csv['StudyInstanceUID'].nunique())

5970
5970


In [ ]:
study_csv.head()

,id,Negative for Pneumonia,Typical Appearance,Indeterminate Appearance,Atypical Appearance
0,00086460a852_study,0,1,0,0
2,00292f8c37bd_study,1,0,0,0
3,005057b3f880_study,1,0,0,0
4,0051d9b12e72_study,0,0,0,1
5,00792b5c8852_study,1,0,0,0


In [ ]:
study_csv.sum()

id                          00086460a852_study00292f8c37bd_study005057b3f8...
Negative for Pneumonia                                                   1676
Typical Appearance                                                       2854
Indeterminate Appearance                                                 1049
Atypical Appearance                                                       391
dtype: object

In [ ]:
study_csv.reset_index(drop=True, inplace=True)

In [ ]:
# study 중에 atypical 인 study 삭제
for i in range(len(study_csv)):
    study_data = study_csv.loc[i]['Atypical Appearance']
    
    if study_data == 1:
        study_csv = study_csv.drop(i)

In [ ]:
study_csv.reset_index(drop=True, inplace=True)

,id,Negative for Pneumonia,Typical Appearance,Indeterminate Appearance,Atypical Appearance
0,00086460a852_study,0,1,0,0
1,00292f8c37bd_study,1,0,0,0
2,005057b3f880_study,1,0,0,0
4,00792b5c8852_study,1,0,0,0
5,00908ffd2d08_study,1,0,0,0
...,...,...,...,...,...
5965,ffcb4630f46f_study,0,1,0,0
5966,ffe4d6e8fbb0_study,0,1,0,0
5967,ffe94fcb14fa_study,0,1,0,0
5968,ffebf1ef4a9c_study,0,1,0,0


In [ ]:
study_csv.to_csv('train_study_level.csv', mode='w', index=False)
image_csv.to_csv('train_image_level.csv', mode='w', index=False)